In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [39]:
# import csv data into dataframes
books_filename = 'book-crossings/BX-Books.csv'
ratings_filename = 'book-crossings/BX-Book-Ratings.csv'

df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})


user_counts = df_ratings['user'].value_counts()
isbn_counts = df_ratings['isbn'].value_counts()

# filter out users who have rated less than 200 books
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# filter out books that have been rated less than 100 times
df_ratings = df_ratings[df_ratings['isbn'].isin(isbn_counts[isbn_counts >= 100].index)]

# merge the two dataframes
df = pd.merge(df_ratings, df_books, on='isbn')

df.drop_duplicates(subset=["title", "user"], inplace=True)

# create a pivot table
df_pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)

# convert the pivot table to a sparse matrix
mat_books = csr_matrix(df_pivot.values)
mat_books

# create a nearest neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(mat_books)

# function to return recommended books
def get_recommends(book = ""):
    # get the index of the book
    book_index = df_pivot.index.get_loc(book)
    distances, indices = model_knn.kneighbors(df_pivot.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append([df_pivot.index[indices.flatten()[i]], distances.flatten()[i]])
    return [book, recommended_books[-1::-1]]

get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")


["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.8016211],
  ['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075],
  ['The Lovely Bones: A Novel', 0.7234864]]]

In [40]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
